In [63]:
import json
import pandas as pd
from openai import OpenAI
from dotenv import dotenv_values
from pycaret.clustering import predict_model, load_model
import ollama


In [64]:
env = dotenv_values(".env")

openai_client = OpenAI(api_key=env["OPENAI_API_KEY"])

ładujemy nasze dane

In [65]:
df = pd.read_csv('welcome_survey_simple_v2.csv', sep=';')
df.head()

,age,edu_level,fav_animals,fav_place,gender
0,<18,Podstawowe,Brak ulubionych,NaN,Kobieta
1,25-34,Średnie,Psy,Nad wodą,Mężczyzna
2,45-54,Wyższe,Psy,W lesie,Mężczyzna
3,35-44,Średnie,Koty,W górach,Mężczyzna
4,35-44,Wyższe,Psy,Nad wodą,Mężczyzna


ładujemy model wytrenowany w poprzedniej lekcji

In [66]:
kmeans_pipeline = load_model('welcome_survey_clustering_pipeline_v2')

Transformation Pipeline and Model Successfully Loaded


aplikujemy model do danych

In [91]:
df_with_clusters = predict_model(model=kmeans_pipeline, data=df)
df_with_clusters["Cluster"].value_counts()

Cluster 1    52
Cluster 0    44
Cluster 3    29
Cluster 2    27
Cluster 6    26
Cluster 7    20
Cluster 4    18
Cluster 5    13
Name: Cluster, dtype: int64

stworzymy teraz prompt, który prześlemy do LLM-a w celu znalezienia odpowiednich nazw i opisów dla klastrów

In [94]:
cluster_descriptions = {}
for cluster_id in df_with_clusters['Cluster'].unique():
    cluster_df = df_with_clusters[df_with_clusters['Cluster'] == cluster_id]
    summary = ""
    for column in df_with_clusters:
        if column == 'Cluster':
            continue

        value_counts = cluster_df[column].value_counts()
        value_counts_str = ', '.join([f"{idx}: {cnt}" for idx, cnt in value_counts.items()])
        summary += f"{column} - {value_counts_str}\n"

    cluster_descriptions[cluster_id] = summary

In [95]:
print(cluster_descriptions["Cluster 5"])

age - 55-64: 12, 18-24: 1, 25-34: 0, 35-44: 0, 45-54: 0, <18: 0, >=65: 0, unknown: 0
edu_level - Wyższe: 13, Podstawowe: 0, Średnie: 0
fav_animals - Psy: 7, Inne: 4, Koty: 1, Koty i Psy: 1, Brak ulubionych: 0
fav_place - Nad wodą: 6, W lesie: 2, Inne: 0, W górach: 0
gender - Mężczyzna: 13, Kobieta: 0



In [81]:
prompt = "Użyliśmy algorytmu klastrowania."
for cluster_id, description in cluster_descriptions.items():
    prompt += f"\n\nKlaster {cluster_id}:\n{description}"

prompt += """
Wygeneruj najlepsze nazwy dla każdego z klasterów oraz ich opisy

Użyj formatu JSON. Przykładowo:
{
    "Cluster 0": {
        "name": "Klaster 0",
        "description": "W tym klastrze znajdują się osoby, które..."
    },
    "Cluster 1": {
        "name": "Klaster 1",
        "description": "W tym klastrze znajdują się osoby, które..."
    }
}
"""
print(prompt)

Użyliśmy algorytmu klastrowania.

Klaster Cluster 2:
age - 25-34: 8, 35-44: 8, 45-54: 6, 18-24: 3, <18: 1, >=65: 1, 55-64: 0, unknown: 0
edu_level - Średnie: 23, Wyższe: 3, Podstawowe: 1
fav_animals - Psy: 13, Koty: 11, Brak ulubionych: 2, Inne: 1, Koty i Psy: 0
fav_place - Nad wodą: 13, W lesie: 6, Inne: 1, W górach: 0
gender - Mężczyzna: 21, Kobieta: 6


Klaster Cluster 0:
age - 45-54: 21, 25-34: 12, 35-44: 10, 55-64: 1, 18-24: 0, <18: 0, >=65: 0, unknown: 0
edu_level - Wyższe: 44, Podstawowe: 0, Średnie: 0
fav_animals - Psy: 44, Brak ulubionych: 0, Inne: 0, Koty: 0, Koty i Psy: 0
fav_place - W górach: 27, W lesie: 15, Inne: 2, Nad wodą: 0
gender - Mężczyzna: 31, Kobieta: 12


Klaster Cluster 7:
age - 45-54: 6, 18-24: 5, 25-34: 5, 35-44: 4, 55-64: 0, <18: 0, >=65: 0, unknown: 0
edu_level - Średnie: 20, Podstawowe: 0, Wyższe: 0
fav_animals - Psy: 7, Inne: 6, Koty: 4, Brak ulubionych: 3, Koty i Psy: 0
fav_place - W górach: 18, Inne: 2, Nad wodą: 0, W lesie: 0
gender - Mężczyzna: 18, Ko

In [86]:
# Initialize Ollama
ollama_host = "localhost"
ollama_port = 11434  
ollama_client = ollama.Client(host=f"{ollama_host}:{ollama_port}")

models = ollama_client.list()
model = models["models"][1]["name"]

sys_msg = "Jesteś asystentem, który pomaga w analizie danych. Zawsze dopowiadasz w formacie JSON, bez dodatkowych komentarzy."
options = {"temperature": 0.1}

In [87]:
ollama_response = ollama_client.generate(model=model, options = options,
                              system=sys_msg,prompt=prompt,
                            #   suffix=suffix,
                              )["response"]
                                
print(ollama_response)

{
    "Cluster 2": {
        "name": "Młodzi miłośnicy zwierząt",
        "description": "W tym klastrze znajdują się głównie młode osoby (25-34 lat), które preferują psy i koty jako ulubione zwierzęta. Większość z nich to mężczyźni."
    },
    "Cluster 0": {
        "name": "Miłośnicy gór",
        "description": "W tym klastrze znajdują się osoby, które preferują psy jako ulubione zwierzęta i lubią spędzać czas w górach. Większość z nich to wykształceni mężczyźni."
    },
    "Cluster 6": {
        "name": "Miłośnicy nadmorskich krajobrazów",
        "description": "W tym klastrze znajdują się głównie wykształceni mężczyźni w wieku 45-54 lat, którzy preferują psy i lubią spędzać czas nad wodą."
    },
    "Cluster 4": {
        "name": "Miłośnicy lasów",
        "description": "W tym klastrze znajdują się głównie wykształceni mężczyźni w wieku 45-54 lat, którzy preferują inne zwierzęta i lubią spędzać czas w lesie."
    },
    "Cluster 5": {
        "name": "Miłośnicy nadmorskich kr

In [80]:
cluster_names_and_descriptions = json.loads(ollama_response)
with open("welcome_survey_cluster_names_and_descriptions_v2o.json", "w") as f:
    f.write(json.dumps(cluster_names_and_descriptions))

In [74]:
response = openai_client.chat.completions.create(
    model="gpt-4o",
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": [{"type": "text", "text": prompt}],
        }
    ],
)

In [75]:
result = response.choices[0].message.content.replace("```json", "").replace("```", "").strip()
cluster_names_and_descriptions = json.loads(result)

In [76]:
with open("welcome_survey_cluster_names_and_descriptions_v2.json", "w") as f:
    f.write(json.dumps(cluster_names_and_descriptions))

In [77]:
with open("welcome_survey_cluster_names_and_descriptions_v2.json", "r") as f:
    print(json.loads(f.read()))

{'Cluster 0': {'name': 'Profesjonaliści górscy', 'description': 'W tym klastrze znajdują się osoby, które są w wieku 45-54 lat, z wyższym wykształceniem, preferujące psy jako ulubione zwierzęta i lubiące spędzać czas w górach. Większość z nich to mężczyźni.'}, 'Cluster 1': {'name': 'Młodzi profesjonaliści nad wodą', 'description': 'W tym klastrze znajdują się osoby głównie w wieku 35-44 lat, z wyższym wykształceniem, preferujące psy i lubiące spędzać czas nad wodą. Większość z nich to mężczyźni.'}, 'Cluster 2': {'name': 'Średnio wykształceni miłośnicy zwierząt', 'description': 'W tym klastrze znajdują się osoby w różnym wieku, głównie z średnim wykształceniem, które preferują zarówno psy, jak i koty, oraz lubią spędzać czas nad wodą. Większość z nich to mężczyźni.'}, 'Cluster 3': {'name': 'Miłośnicy kotów w górach', 'description': 'W tym klastrze znajdują się osoby w wieku 35-44 lat, z wyższym wykształceniem, które preferują koty i lubią spędzać czas w górach. Większość z nich to mężcz